**Twitter Disaster Prediction** <br>

Submitted by <br>
* Haigal Harrison - C0816642



roBERTa model from tensorflow is used to predict whether a tweet is about a real disaster or not

In [ ]:
# Importing the required libraries
import pandas as pd
import numpy as np
from google.colab import drive

In [ ]:
# Mounting Drive
drive.mount('/content/drive/',force_remount=True)

Mounted at /content/drive/


In [ ]:
# importing the required tensorflow libraries
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub

In [ ]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [ ]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 5.2 MB/s 


In [ ]:
import tokenization

In [ ]:
# a seed is set of better reproducability
SEED = 1002
def seed_everything(seed):
    np.random.seed(seed)
    tf.random.set_seed(seed) 
    
seed_everything(SEED)

In [ ]:
# reading the training data from the google drive using pandas
train_df = pd.read_csv('/content/drive/My Drive/My data files/twitter_disaster_data.csv')
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


Bert tokenization method is used to tokenize the tweets collected via twitter API. It includes the following 3 steps,

* Text Normalization - The text is converted to lowercase , all the whitespaces are converted to spaces and get rid of accent markers

* Punctuation Splitting - A space is added to each side of the punctuation marks

* It is also called whitespace tokenization and applies WordPiece tokenization to every word

Finally it reduces the length of the text to the given length we have provided and add [CLS] and [SEP] tags to the end. [CLS] indicates the start and [SEP] indicates the end of each sentence. <br>
Tokenizer method is used to convert words to integers. An input mask and segments ids are also created.

In [ ]:
def encode_bert(txt, tokenizer, max_length=512):
  tokens = []
  masks = []
  segments = []
  for t in txt:
    t = tokenizer.tokenize(t)
    t = t[:max_length-2]
    inp_seq = ["[CLS]"] + t + ["[SEP]"]
    pad_length = max_length - len(inp_seq)

    token = tokenizer.convert_tokens_to_ids(inp_seq)
    token += [0] * pad_length
    pad_masks = [1] * len(inp_seq) + [0] * pad_length
    seg_ids = [0] * max_length

    tokens.append(token)
    masks.append(pad_masks)
    segments.append(seg_ids)
    
  return np.array(tokens), np.array(masks), np.array(segments)

Plain text input is converted to the expected input format by the roBERTa Model. <br>

* input_word_ids : Maps each words to its corresponding token id.
* input_mask : Specified the start and end of a sentence using an array. No padding tokens are given a value of 1 and padding tokens are given a value of 0.
* segment_ids : Recognizes the segments of the sentences

In [ ]:
def model_build(bert_layer, max_length=512):
  inp_word_ids = Input(shape=(max_length,), dtype=tf.int32, name="inp_word_ids")
  inp_mask = Input(shape=(max_length,), dtype=tf.int32, name="inp_mask")
  seg_ids = Input(shape=(max_length,), dtype=tf.int32, name="seg_ids")

  _, sequence_output = bert_layer([inp_word_ids, inp_mask, seg_ids])
  clf_output = sequence_output[:, 0, :]
  out = Dense(1, activation='sigmoid')(clf_output)

  model = Model(inputs=[inp_word_ids, inp_mask, seg_ids], outputs=out)
    
  model.compile(Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
  return model


Loads the roBERTa model. Large uncased model is used. It is set as a Keras Layer to use it to our specific case.

In [ ]:
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

Tokenizer converts the input data to a format BERT understands. Vocab file is used so that tokenizer knows which number to be used to encode each words.

In [ ]:
voc_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()

#returns true/false based on selected cased/uncased bert layer
lower_case = bert_layer.resolved_object.do_lower_case.numpy()

#Initialize the tokenizer
tokenizer = tokenization.FullTokenizer(voc_file, lower_case)

In [ ]:
train_inp = encode_bert(train_df.text.values, tokenizer, max_length=160)
train_label = train_df.target.values

The model summary shows three input layers we created followed by roBERTa model in the Keras layer. Last dense layer predicts the output in a scale of 0-1

In [ ]:
model = model_build(bert_layer, max_length=160)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inp_word_ids (InputLayer)      [(None, 160)]        0           []                               
                                                                                                  
 inp_mask (InputLayer)          [(None, 160)]        0           []                               
                                                                                                  
 seg_ids (InputLayer)           [(None, 160)]        0           []                               
                                                                                                  
 keras_layer (KerasLayer)       [(None, 1024),       335141889   ['inp_word_ids[0][0]',           
                                 (None, 160, 1024)]               'inp_mask[0][0]',           

Model is trained using Keras. ModelCheckpoint is used to save the best model with high validation accuracy

In [ ]:
model_checkpoint = ModelCheckpoint('model.h5', monitor='val_accuracy', save_best_only=True)
history = model.fit(
    train_inp, train_label,
    validation_split=0.1,
    epochs=3,
    callbacks=[model_checkpoint],
    batch_size=5
)

Epoch 1/3
1371/1371 [==============================] - 1667s 1s/step - loss: 0.4232 - accuracy: 0.8234 - val_loss: 0.3645 - val_accuracy: 0.8425
Epoch 2/3
1371/1371 [==============================] - 1621s 1s/step - loss: 0.2655 - accuracy: 0.8981 - val_loss: 0.4105 - val_accuracy: 0.8504
Epoch 3/3
1371/1371 [==============================] - 1556s 1s/step - loss: 0.1586 - accuracy: 0.9399 - val_loss: 0.5257 - val_accuracy: 0.8360


Model is saved to google drive so that it can be used to predict tweets in realtime.

In [ ]:
model.save('/content/drive/My Drive/My data files/twitter_model')

INFO:tensorflow:Assets written to: /content/drive/My Drive/My data files/twitter_model/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/My data files/twitter_model/assets
